In [ ]:
%reload_ext autoreload
%autoreload 2

import time
import itertools
import numpy as np
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook

from optimization import *
from optimal_solution import *


In [ ]:
def plotResult(duration, selected_cities_n, selected_cities_n_convex, loss_values, 
               loss_value_convex, num_cities_per_step):
    print("d= %s seconds" % duration)
    if loss_value_convex is not None:
        # be careful, at the end we want to return '-final loss'
        print('Final loss ' + str(loss_values[-2]))
        print('Final loss with Convex Hull ' + str(loss_value_convex))
    else:
        print('Final loss ' + str(loss_values[-1]))
    # print(selected_cities_n)

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    # fig.suptitle('Results')
    
    print(np.sum(selected_cities_n))
    axes[0].plot(loss_values)
    axes[0].set_ylabel('Loss')
    axes[0].set_xlabel('Iterations')
    axes[0].set_title('Loss Evolution')
    m = selected_cities_n == 1
    not_selected = selected_cities_n_convex == 0
    axes[1].scatter(g.x[not_selected, 0], g.x[not_selected, 1], label='Non selected Cities')
    axes[1].scatter(g.x[m, 0], g.x[m, 1], c='r', label='Selected cities')
    if selected_cities_n_convex is not None:
        mbis = (selected_cities_n_convex == 1) & (selected_cities_n == 0)
        mter = (selected_cities_n_convex == 0) & (selected_cities_n == 1)
        axes[1].scatter(g.x[mbis, 0], g.x[mbis, 1], c='g',
                        label='Added cities (Convex Hull)')
        axes[1].scatter(g.x[mter, 0], g.x[mter, 1], c='y',
                        label='Selected Cities not in Convex Hull')
    axes[1].set_title('Selected cities')
    box = axes[1].get_position()
    axes[1].set_position([box.x0, box.y0, box.width * 0.8, box.height])
    # Put a legend to the right of the current axis
    axes[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.xlim(0, 1)
    plt.ylim(0, 1)

    if num_cities_per_step is not None:
        plt.figure(figsize=(4, 2))
        plt.plot(np.arange(n_iter), num_cities_per_step)
        plt.title("#selected cities in each step")


# np.random.seed(42)
N = 500
g = G2(N)



In [ ]:
# Run the optimization to compute the selected cities

n_iter = 20000

l = 1.5 # lambda
beta = 25
beta = 1


# other options
initial_selection_probability=0.0
mutation_strategy = 1
mutation_strategy = 4

# def beta(i):
#     return np.log(i)


probablities = g.v / np.sum(g.v)
selected_cities_idx = np.random.choice(np.arange(N), 1, p=g.v / np.sum(g.v))
init_selected_cities = np.zeros(N, dtype=np.int32)
init_selected_cities[selected_cities_idx] = 1

for mutation_strategy in [1, 3, 4, 5]:

    def beta(i):
        p1 = 0.5
        p50 = 0.006
        beta0 = -np.log(p1)
        beta1 = -np.log(p50)
        v = beta0 * (beta1 / beta0) ** (i/(n_iter-1.0)) 
        return v *4

    if mutation_strategy == 5:
        
        def beta(i):
            
            if i < n_iter // 2: 
                return 0.4
            elif i < n_iter * 2/ 3:
                return 1
            else: 
                return 5
    
    start_time = time.time()
    selected_cities_n, selected_cities_n_convex, loss_values, loss_value_convex = optimize(
        g, l, 
        beta=beta, 
        n_iter=n_iter, 
        mutation_strategy=mutation_strategy,
        initial_selection_probability=initial_selection_probability,
        selected_cities=init_selected_cities)
    
    loss_convex_pass_one = loss_value_convex
    # Second pass
    if mutation_strategy == 5: 
        selected_cities_n, selected_cities_n_convex, loss_values, loss_value_convex = optimize(
            g, l, 
            beta=35, 
            n_iter=n_iter, 
            mutation_strategy=4,
            initial_selection_probability=initial_selection_probability,
            selected_cities=selected_cities_n_convex)
        print(f"Second pass changed loss from {loss_convex_pass_one} to {loss_value_convex}")

        

    d = (time.time() - start_time)
    num_cities_per_step = np.array(selected_cities_n).sum(axis=1)
    if type(selected_cities_n) == list:
        selected_cities_n = selected_cities_n[-1]

    loss_values[-1] = (loss_value_convex)
    plotResult(d, selected_cities_n, selected_cities_n_convex,
               loss_values, loss_value_convex, num_cities_per_step)
